In [1]:
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [2]:
train_data = pd.read_csv("../Archive/people_train.csv")
train_data.head()

,Unnamed: 0,suspicious,turnover,transaction_count,io_ratio,age,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,...,distinct_counterparties_log,transaction_in,transaction_out,transaction_in_log,transaction_out_log,transaction_count_log,0,1,2,3
0,0,0,0.143663,-0.351520,0.142857,1.846564,0,-0.421181,0.108757,1.005063,...,1.039112,-0.432306,-0.312382,-0.721217,0.260517,0.183046,0,0,1,0
1,2,0,-0.113321,-0.450280,0.666667,1.736781,0,-0.212764,-0.544194,0.003134,...,-0.615425,-0.352255,-0.489128,-0.174106,-0.888673,-0.330729,0,0,1,0
2,3,0,-0.106171,1.475538,0.308642,1.736781,0,-0.844768,-0.255389,0.003134,...,0.024637,1.488931,1.455079,1.819491,1.693087,1.667765,0,0,1,0
3,4,0,-0.044296,-0.499660,1.000000,-0.349103,0,1.884350,0.410118,2.006993,...,-0.615425,-0.432306,-0.524477,-0.721217,-0.888673,-0.996894,0,1,0,0
4,5,0,-0.117145,-0.450280,0.333333,-0.513778,0,-0.046556,-0.318172,0.003134,...,0.399049,-0.432306,-0.453778,-0.721217,-0.444106,-0.330729,0,1,0,0


In [3]:
val_data = pd.read_csv("../Archive/people_validate.csv")
val_data.head()

,Unnamed: 0,suspicious,turnover,transaction_count,io_ratio,age,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,...,distinct_counterparties_log,transaction_in,transaction_out,transaction_in_log,transaction_out_log,transaction_count_log,0,1,2,3
0,17,0,0.258922,-0.499660,1.000000,1.078080,0,0.757848,0.749150,1.005063,...,-0.615425,-0.432306,-0.524477,-0.721217,-0.888673,-0.996894,0,0,1,0
1,21,0,-0.116272,-0.005861,0.190476,1.462322,0,-0.740466,-0.293059,-0.998796,...,0.024637,-0.192152,0.076460,0.373006,0.928479,0.849211,0,0,1,0
2,25,0,-0.059747,5.154346,0.282609,-0.678453,0,-0.865404,-0.142378,0.003134,...,3.101643,4.690994,5.308143,2.573691,2.386152,2.300590,0,1,0,0
3,28,0,-0.115467,-0.474970,0.000000,1.352539,0,1.525107,1.778803,-0.998796,...,-0.615425,-0.512358,-0.453778,-0.721217,-0.444106,-0.576591,0,0,1,0
4,34,0,-0.087982,-0.474970,0.000000,0.364489,0,0.384911,0.259438,-0.998796,...,-0.615425,-0.512358,-0.453778,-0.721217,-0.444106,-0.576591,0,0,1,0


In [4]:
test_data = pd.read_csv("../people_test.csv").reset_index()
original_test = pd.read_csv("../test.csv")
test_data["customer"] = original_test[original_test["category"] == 0].reset_index()["customer"]
test_data.head()

,index,Unnamed: 0,turnover,transaction_count,io_ratio,age,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,...,transaction_in,transaction_out,transaction_in_log,transaction_out_log,transaction_count_log,0,1,2,3,customer
0,0,0,0.438708,-1.095244,0.00,1.023189,0,1.145452,0.773999,-43.443527,...,-3.631127,-1.568086,-35.803235,-29.092674,-27.504824,0,0,1,0,91000000
1,1,2,0.007795,-1.021174,0.25,-0.239320,0,-0.270508,0.208946,-43.443527,...,-3.551076,-1.497388,-35.803235,-28.388051,-26.664217,0,1,0,0,91000002
2,2,3,0.025880,-1.021174,0.50,-1.611612,0,-0.397134,0.648432,-43.443527,...,-3.471024,-1.532737,-35.256123,-28.648106,-26.664217,1,0,0,0,91000003
3,3,4,0.003521,-1.095244,0.00,0.035139,0,0.046526,0.033152,-43.443527,...,-3.631127,-1.568086,-35.803235,-29.092674,-27.504824,0,1,0,0,91000004
4,4,5,0.001434,-0.996484,0.00,1.956347,0,-0.321909,0.849339,-40.437738,...,-3.631127,-1.426689,-35.803235,-28.060421,-26.528910,0,0,1,0,91000005


In [5]:
train_features = train_data.drop(["Unnamed: 0", "suspicious"], axis=1).values
train_labels = train_data["suspicious"].values

In [6]:
val_features = val_data.drop(["Unnamed: 0", "suspicious"], axis=1).values
val_labels = val_data["suspicious"].values

In [7]:
train_features.shape

(640110, 41)

In [8]:
train_features[train_labels == 1].shape

(6448, 41)

In [9]:
augmented_train_features = train_features
augmented_train_labels = train_labels
for i in range(50):
    augmented_train_features = np.concatenate((augmented_train_features, train_features[train_labels == 1]))
    augmented_train_labels = np.concatenate((augmented_train_labels, np.array([1] * train_features[train_labels == 1].shape[0])))

In [10]:
augmented_train_labels.sum()/len(augmented_train_labels)

0.34165671006015524

In [9]:
knnc = KNeighborsClassifier(n_neighbors=500, n_jobs=4)
knnc.fit(train_features, train_labels) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=4, n_neighbors=500, p=2,
           weights='uniform')

In [10]:
preds = knnc.predict_proba(val_features[val_labels == 1])

In [11]:
preds[:,1].mean()

0.0358974358974359

In [18]:
test_preds = val_features[val_labels == 0]
np.random.shuffle(test_preds)
preds = knnc.predict_proba(test_preds[:10000])

In [19]:
preds[:,1].mean()

0.0077411

In [24]:
val_features.shape

(160036, 41)

In [ ]:
predictions = []
for i in range(len(val_features)//1000):
    to = min(len(val_features), (i+1)*1000)
    predictions.extend(knnc.predict_proba(val_features[i*1000:to,:]))
    print("Progress: " + str(i) + "/" + str(len(val_features)))

Progress: 0/160036
Progress: 1/160036
Progress: 2/160036
Progress: 3/160036
Progress: 4/160036
Progress: 5/160036


In [29]:
def cost(preds, labels, tresh):
    investigation_cost = sum(preds > tresh) * 1000
    fine_cost = sum(labels[preds < tresh]) * 60000
    return investigation_cost + fine_cost

In [30]:
for i in range(10):
    history = model.fit(augmented_train_features, tf.keras.utils.to_categorical(augmented_train_labels, 2), epochs=1, batch_size=256, shuffle=True, validation_data=(val_features,  tf.keras.utils.to_categorical(val_labels, 2)),
                verbose=1, )
    
    min_cost = -1
    min_cost_tresh = 0
    preds = model.predict(val_features)[:,1]
    for i in np.linspace(0, 1, 21):
        c = cost(preds, val_labels, i)
        if min_cost == -1:
            min_cost = c
            
        if c <= min_cost:
            min_cost = c
            min_cost_tresh = i
        
    print("COST: " + str(min_cost) + " for tresh. " + str(min_cost_tresh))
    

Train on 962510 samples, validate on 160036 samples
Epoch 1/1
962510/962510 [==============================] - 10s 10us/step - loss: 0.4315 - precision: 0.7470 - recall: 0.6264 - val_loss: 0.2834 - val_precision: 0.0629 - val_recall: 0.6696
COST: 51004000 for tresh. 0.4
Train on 962510 samples, validate on 160036 samples
Epoch 1/1
962510/962510 [==============================] - 10s 10us/step - loss: 0.4103 - precision: 0.7625 - recall: 0.6522 - val_loss: 0.2693 - val_precision: 0.0657 - val_recall: 0.6691
COST: 49720000 for tresh. 0.45
Train on 962510 samples, validate on 160036 samples
Epoch 1/1
962510/962510 [==============================] - 10s 10us/step - loss: 0.4069 - precision: 0.7635 - recall: 0.6572 - val_loss: 0.2750 - val_precision: 0.0649 - val_recall: 0.6798
COST: 49788000 for tresh. 0.5
Train on 962510 samples, validate on 160036 samples
Epoch 1/1
962510/962510 [==============================] - 10s 10us/step - loss: 0.4048 - precision: 0.7641 - recall: 0.6609 - val_los

In [46]:
full_train_data = pd.concat((train_data, val_data), axis=0)
len(full_train_data)

800146

In [47]:
full_train_features = full_train_data.drop(["Unnamed: 0", "suspicious"], axis=1).values
full_train_labels = full_train_data["suspicious"].values

In [50]:
full_augmented_train_features = full_train_features
full_augmented_train_labels = full_train_labels
for i in range(50):
    full_augmented_train_features = np.concatenate((full_augmented_train_features, full_train_features[full_train_labels == 1]))
    full_augmented_train_labels = np.concatenate((full_augmented_train_labels, np.array([1] * full_train_features[full_train_labels == 1].shape[0])))

In [51]:
full_augmented_train_labels.sum()/len(full_augmented_train_labels)

0.34348174231346923

In [52]:
encoding_dim = train_features.shape[-1]
model_input = keras.models.Input(shape=(encoding_dim,))
x = Dense(32, activation='tanh')(model_input)
x = keras.layers.Dropout(0.1)(x)
x = Dense(16, activation='tanh')(x)
x = keras.layers.Dropout(0.1)(x)
#x = Dense(8, activation='tanh')(x)
#x = keras.layers.Dropout(0.1)(x)
out = Dense(2, activation='sigmoid')(x)

model = Model(model_input, out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras_metrics.precision(), keras_metrics.recall()])

In [53]:
for i in range(10):
    history = model.fit(full_augmented_train_features, tf.keras.utils.to_categorical(full_augmented_train_labels, 2), epochs=1, batch_size=256, shuffle=True,
                verbose=1, )   

Exception ignored in: <bound method ScopedTFStatus.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFStatus object at 0x7f3db0107cc0>>
Traceback (most recent call last):
  File "/home/nathan/anaconda3/envs/ipykernel_py3_6/lib/python3.6/site-packages/tensorflow/python/framework/c_api_util.py", line 39, in __del__
    c_api.TF_DeleteStatus(self.status)
AttributeError: 'ScopedTFStatus' object has no attribute 'status'


Epoch 1/1
1206396/1206396 [==============================] - 13s 11us/step - loss: 0.4308 - precision: 0.7497 - recall: 0.6233
Epoch 1/1
1206396/1206396 [==============================] - 13s 10us/step - loss: 0.4113 - precision: 0.7637 - recall: 0.6520
Epoch 1/1
1206396/1206396 [==============================] - 12s 10us/step - loss: 0.4063 - precision: 0.7655 - recall: 0.6583
Epoch 1/1
1206396/1206396 [==============================] - 13s 10us/step - loss: 0.4042 - precision: 0.7660 - recall: 0.6635
Epoch 1/1
1206396/1206396 [==============================] - 12s 10us/step - loss: 0.4027 - precision: 0.7665 - recall: 0.6653
Epoch 1/1
1206396/1206396 [==============================] - 13s 10us/step - loss: 0.4021 - precision: 0.7666 - recall: 0.6671
Epoch 1/1
1206396/1206396 [==============================] - 13s 11us/step - loss: 0.4012 - precision: 0.7671 - recall: 0.6678
Epoch 1/1
1206396/1206396 [==============================] - 13s 10us/step - loss: 0.4006 - precision: 0.7673 -

In [70]:
val = model.predict(train_data.drop(["Unnamed: 0","suspicious"], axis=1).values)[:,1]

In [71]:
val.mean()

0.20287384

In [72]:
val.std()

0.21415697

In [117]:
test_data["predictions"] = model.predict(test_data.drop(["Unnamed: 0","customer","index"], axis=1).values)[:,1]

In [138]:
len(test_data[test_data["predictions"] >= 0.5])

580926

In [139]:
test_data[test_data["predictions"] >= 0.5]

,index,Unnamed: 0,turnover,transaction_count,io_ratio,age,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,...,transaction_out,transaction_in_log,transaction_out_log,transaction_count_log,0,1,2,3,customer,predictions
0,0,0,0.438708,-1.095244,0.000000,1.023189,0,1.145452,0.773999,-43.443527,...,-1.568086,-35.803235,-29.092674,-27.504824,0,0,1,0,91000000,0.822518
3,3,4,0.003521,-1.095244,0.000000,0.035139,0,0.046526,0.033152,-43.443527,...,-1.568086,-35.803235,-29.092674,-27.504824,0,1,0,0,91000004,0.538316
4,4,5,0.001434,-0.996484,0.000000,1.956347,0,-0.321909,0.849339,-40.437738,...,-1.426689,-35.803235,-28.060421,-26.528910,0,0,1,0,91000005,0.776352
5,5,6,0.875894,-1.070554,0.000000,0.254705,0,0.318162,0.221503,-43.443527,...,-1.532737,-35.803235,-28.648106,-27.084521,0,0,1,0,91000006,0.755813
6,6,7,0.193250,-1.095244,0.000000,-1.172478,0,0.947540,1.841322,-43.443527,...,-1.568086,-35.803235,-29.092674,-27.504824,0,1,0,0,91000007,0.526884
7,7,8,0.007438,-1.070554,0.500000,0.803622,0,0.557345,0.384740,-43.443527,...,-1.568086,-35.803235,-29.092674,-27.084521,0,0,1,0,91000008,0.802373
8,8,11,0.002491,-1.095244,1.000000,0.254705,0,-0.328287,-0.217983,-43.443527,...,-1.603435,-35.803235,-29.092674,-27.504824,0,0,1,0,91000011,0.635883
9,9,12,0.027425,-0.848344,0.272727,2.066131,0,-0.593732,0.121049,-44.445457,...,-1.320642,-34.936083,-27.758972,-26.050814,0,0,1,0,91000012,0.807840
10,10,13,0.006401,-0.996484,0.400000,0.638947,0,-0.486616,-0.318437,-43.443527,...,-1.497388,-35.256123,-28.388051,-26.528910,0,0,1,0,91000013,0.770919
12,12,15,0.012508,-1.095244,0.000000,-0.349103,0,-0.128687,-0.079859,-43.443527,...,-1.568086,-35.803235,-29.092674,-27.504824,0,1,0,0,91000015,0.528927


In [140]:
len(test_data[test_data["predictions"] >= 0.5].reset_index()[["customer"]])

580926

In [141]:
test_data[test_data["predictions"] >= 0.5].reset_index()[["customer"]].to_csv("../cust_nr_nn.csv", index=False)